Import the necessary dependent libraries

In [1]:
import os
from module import MLPModule
from utils import aggregate_weights, save_global_measure, load_client_data, calculate_global_measures
from tqdm import tqdm
import torch
from client import Client

Set the necessary parameters and hyperparameters for the experiment and model

In [2]:
seed = 54
dataset_name = "unsw-nb15"
num_rounds = 25
input_dim = 0

if torch.cuda.is_available():
    device = torch.device("cuda")
# elif torch.backends.mps.is_available():
#     device = torch.device("mps:0")
else:
    device = torch.device("cpu")

hyperparameters = {
    'input_dim': input_dim,
    'lr': 0.001,
    'hidden_neurons_num': 512,
    'batch_size': 128,
    'seed': seed,
    'device': device
}

In [3]:
client_data_list = load_client_data(os.path.join(dataset_name, "split"), device)

In [4]:
def process_client_data(data_list, keep_columns):
    processed_data_list = []

    for client_data in data_list:
        X_train, Y_train = client_data[0]
        processed_X_train = X_train[:, keep_columns]
        processed_train = (processed_X_train, Y_train)

        X_val, Y_val = client_data[1]
        processed_X_val = X_val[:, keep_columns]
        processed_val = (processed_X_val, Y_val)

        X_test, Y_test = client_data[2]
        processed_X_test = X_test[:, keep_columns]
        processed_test = (processed_X_test, Y_test)

        processed_client_data = (processed_train, processed_val, processed_test)
        processed_data_list.append(processed_client_data)

    return processed_data_list

In [5]:
columns_to_keep = [0, 2, 4, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 28, 29, 38, 39]
if dataset_name == "n-baiot":
    columns_to_keep = [0, 3, 6, 30, 37, 44, 31, 38, 45, 32, 39, 46, 60, 63, 66, 75, 82, 89, 76, 83, 90]
client_data_list = process_client_data(client_data_list, columns_to_keep)
hyperparameters['input_dim'] = len(columns_to_keep)

Generate clients

In [6]:
def create_clients(client_data, hyperparameter):
    client_list = []
    for i, (train_set, test_set, val_set) in enumerate(client_data):
        c = Client(
            id=i,
            hyperparameters=hyperparameter,
            train_set=train_set,
            test_set=test_set,
            val_set=val_set
        )
        client_list.append(c)
    return client_list
clients = create_clients(client_data_list, hyperparameters)

FedAvg training process

In [7]:
# Initialize model
global_model = MLPModule(hyperparameters)

# Pre-trained
model_weights_list = []
client_n_samples_list = {}
for client in clients:
    client.receive_global_model(global_model)
    client_model_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
    client_n_samples_list[client.id] = client_n_samples
    model_weights_list.append(client_model_parameters)
    
global_model.set_parameters(aggregate_weights(model_weights_list, client_n_samples_list))

In [8]:
# Record training results
record_global_train_measures = [None] * num_rounds
record_global_val_measures = [None] * num_rounds

pbar = tqdm(total=num_rounds)
# Perform num_rounds rounds of training
for idx in range(num_rounds):
    model_weights_list = []
    client_n_samples_list = {}
    all_client_train_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }
    all_client_val_measures = { 'accuracy': {}, 'fpr': {}, 'tpr': {}, 'ber': {}, 'loss': {} }
    
    for client in clients:
        # Client receives new global model
        client.receive_global_model(global_model)

        # The client trains the model locally
        client_model_parameters, client_measures, client_n_samples = client.local_fit_and_upload_parameters()
        client_n_samples_list[client.id] = client_n_samples
        model_weights_list.append(client_model_parameters)
        for key in all_client_train_measures:
            all_client_train_measures[key][client.id] = (client_measures['train'][key])
            all_client_val_measures[key][client.id] = (client_measures['val'][key])

    global_train_measures, global_val_measures = calculate_global_measures(clients,
                                                                           all_client_train_measures,
                                                                           all_client_val_measures,
                                                                           display_result=True)
    record_global_train_measures[idx] = global_train_measures
    record_global_val_measures[idx] = global_val_measures

    pbar.update(1)

    # Aggregate new global model
    global_model.set_parameters(aggregate_weights(model_weights_list, client_n_samples_list))
    
pbar.reset()

  4%|▍         | 1/25 [00:13<05:28, 13.70s/it]

Global Train Set Loss: 0.3996258521731434
Global Train Set Accuracy: 0.8272163144367685
Global Train Set FPR: 0.14281994768518083
Global Train Set TPR: 0.27600130164572706
Global Train Set BER: 0.4284093230197268
Global Validation Set Loss: 0.3157757683821993
Global Validation Set Accuracy: 0.8580285725913508
Global Validation Set FPR: 0.04747662970694575
Global Validation Set TPR: 0.30399923825843767
Global Validation Set BER: 0.2617386957242541


  8%|▊         | 2/25 [00:27<05:14, 13.67s/it]

Global Train Set Loss: 0.36509207318031534
Global Train Set Accuracy: 0.8330123014877523
Global Train Set FPR: 0.1444068370324117
Global Train Set TPR: 0.3031451460711775
Global Train Set BER: 0.41563084548061724
Global Validation Set Loss: 0.27932165986643365
Global Validation Set Accuracy: 0.8644856985398723
Global Validation Set FPR: 0.038813899221871624
Global Validation Set TPR: 0.339988874192472
Global Validation Set BER: 0.23941251251469986


 12%|█▏        | 3/25 [00:41<05:01, 13.72s/it]

...

Global Train Set Loss: 0.34012716344220684
Global Train Set Accuracy: 0.8871990456900265
Global Train Set FPR: 0.15012579568720216
Global Train Set TPR: 0.42040663345124946
Global Train Set BER: 0.3598595811179763
Global Validation Set Loss: 0.2605209383747952
Global Validation Set Accuracy: 0.9240548824570419
Global Validation Set FPR: 0.018608957473854933
Global Validation Set TPR: 0.4825370801087508
Global Validation Set BER: 0.1580359386825521


 96%|█████████▌| 24/25 [05:32<00:13, 13.82s/it]

Global Train Set Loss: 0.3402873408680388
Global Train Set Accuracy: 0.8874314246447468
Global Train Set FPR: 0.14987928359858077
Global Train Set TPR: 0.42094682030447444
Global Train Set BER: 0.35946623164705316
Global Validation Set Loss: 0.26102198483165673
Global Validation Set Accuracy: 0.924381495381253
Global Validation Set FPR: 0.01806519511000287
Global Validation Set TPR: 0.483421479676337
Global Validation Set BER: 0.15732185771683282


  0%|          | 0/25 [00:00<?, ?it/s]         

Global Train Set Loss: 0.33977771320310024
Global Train Set Accuracy: 0.8874850994785081
Global Train Set FPR: 0.14973713966641602
Global Train Set TPR: 0.4214193657947861
Global Train Set BER: 0.359158886935815
Global Validation Set Loss: 0.2609236231286386
Global Validation Set Accuracy: 0.9248847779664762
Global Validation Set FPR: 0.017204335625303328
Global Validation Set TPR: 0.4853973071959786
Global Validation Set BER: 0.15590351421466228


Save experiment results

In [9]:
save_global_measure(record_global_train_measures, "train_measures_"+dataset_name+".csv", "FedTrust")
save_global_measure(record_global_val_measures, "val_measures_"+dataset_name+".csv", "FedTrust")

Saved measures to Experimental_results/FedTrust/train_measures_unsw-nb15.csv
Saved measures to Experimental_results/FedTrust/val_measures_unsw-nb15.csv


Test the performance of the model on the unseen test set

In [10]:
all_client_measures = {
    'accuracy': [],
    'fpr': [],
    'tpr': [],
    'ber': [],
    'loss': []
}
client_n_samples_list = {}
for client in clients:
    client.receive_global_model(global_model)
    client_measures, client_n_samples = client.evaluate_test_set()
    client_n_samples_list[client.id] = client_n_samples

    for key in all_client_measures:
        all_client_measures[key].append(client_measures[key])

global_measures = {}
for key in all_client_measures:
    global_measures[key] = sum([all_client_measures[key][c.id] * client_n_samples_list[c.id] for c in clients]) / sum(client_n_samples_list.values())

print("Global Test Set Loss:", global_measures['loss'])
print("Global Test Set Accuracy:", global_measures['accuracy'])
print("Global Test Set FPR:", global_measures['fpr'])
print("Global Test Set TPR:", global_measures['tpr'])
print("Global Test Set BER:", global_measures['ber'])

Global Test Set Loss: 0.2849678733798533
Global Test Set Accuracy: 0.8855191418167503
Global Test Set FPR: 0.00016290764224234115
Global Test Set TPR: 0.027515634977215864
Global Test Set BER: 0.40550678759715253
